In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [2]:
import numpy as np
import calendar
from helpers import *
from thermal_optimal_path.lattice import partition_function
from thermal_optimal_path.statistics import average_path
from strategies import basic_strategy
import matplotlib.pyplot as plt
from scipy.stats import norm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/elantonino/Anaconda3/envs/gt/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
year = 2024
month = 7
temperature = 0.1
minute = 40

days_in_month = calendar.monthrange(year, month)[1]

In [ ]:
daily_results = {}

for day in range(1, days_in_month + 1):
    day_str = f"{day:02d}"  
    print(f"Processing day: {year}-{month:02d}-{day_str}")
    
    btc_path = f'../data/BTCUSDT/BTCUSDT-1s-{year}-{month:02d}-{day_str}.parquet'
    eth_path = f'../data/ETHUSDT/ETHUSDT-1s-{year}-{month:02d}-{day_str}.parquet'

    try:
        btc_data = pd.read_parquet(btc_path)
        eth_data = pd.read_parquet(eth_path)
    except FileNotFoundError:
        print(f"Data for {year}-{month:02d}-{day_str} not found, skipping...")
        continue

    btc_prices = btc_data['Open']
    eth_prices = eth_data['Open']
    btc_prices_std = standardize(btc_prices)
    eth_prices_std = standardize(eth_prices)

    btc_returns = btc_prices.pct_change().rolling(window=5).mean()[minute:]
    eth_returns = eth_prices.pct_change().rolling(window=5).mean()[minute:]

    df_avg = []
    weights = np.exp(np.linspace(0, 2, minute))  
    weights /= weights.sum()
    total = len(btc_prices_std)
    for i in range(total - minute):
        end_time = i + minute
        btc = btc_prices_std[i:end_time]
        eth = eth_prices_std[i:end_time]

        g = partition_function(btc, eth, temperature)
        avg_path = average_path(g)[::2]
        weighted_avg = np.dot(avg_path, weights)
        df_avg.append(weighted_avg)

    strategy_result = basic_strategy(
        btc_prices[minute:], 
        btc_returns, 
        eth_prices[minute:], 
        eth_returns, 
        df_avg,
        40
    )

    daily_results[day_str] = {
        "df_avg": df_avg,
        "btc_returns": btc_returns,
        "eth_returns": eth_returns,
        "strategy_result": strategy_result
    }

print("Finished processing all days.")


Processing day: 2024-07-01
Total money after strategy: Base amount + 0.0005185709473746192, Long trades: 5, Short trades: 5, Long returns: 0.000219857213460517, Short returns: 0.00029871373391410216
Processing day: 2024-07-02
Total money after strategy: Base amount + 0.00029155768874411514, Long trades: 3, Short trades: 4, Long returns: -0.00019043798950524327, Short returns: 0.00048199567824958045
Processing day: 2024-07-03
Total money after strategy: Base amount + 0.0002613864534108945, Long trades: 0, Short trades: 2, Long returns: 0, Short returns: 0.0002613864534108945
Processing day: 2024-07-04
Total money after strategy: Base amount + 3.0946137313803135e-05, Long trades: 12, Short trades: 9, Long returns: -0.0010045612417657335, Short returns: 0.0010355073790798697
Processing day: 2024-07-05
Total money after strategy: Base amount + -0.0006213925851178415, Long trades: 7, Short trades: 9, Long returns: -0.0012513117455812406, Short returns: 0.000629919160462955
Processing day: 2

In [7]:
df = pd.read_csv('daily_returns.csv', header=None)
df.columns = ['Daily Returns', 'Long Trades', 'Short Trades', 'Long Returns', 'Short Returns']

df.head()

,Daily Returns,Long Trades,Short Trades,Long Returns,Short Returns
0,0.000519,5,5,0.000220,0.000299
1,0.000292,3,4,-0.000190,0.000482
2,0.000261,0,2,0.000000,0.000261
3,0.000031,12,9,-0.001005,0.001036
4,-0.000391,7,10,-0.001251,0.000861


In [8]:
df.sum(numeric_only=True)

Daily Returns      0.066853
Long Trades      681.000000
Short Trades     644.000000
Long Returns       0.008744
Short Returns      3.057896
dtype: float64